In [1]:
import pandas as pd

In [2]:
""" Get stuff out of Netfile v2 API
"""
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

BASE_URL = 'https://netfile.com/api/campaign'
CONTRIBUTION_FORM = 'F460A'
EXPENDITURE_FORM = 'F460E'

PARAMS = { 'aid': 'COAK' }

def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'API_KEY', 'API_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth

AUTH=get_auth_from_env_file()

pp = PrettyPrinter()

def get_filing(offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    if res.status_code == 500:
        print('ping')
        return get_filing(offset=0)
    else:
        print(res)
        body = res.json()
        results = body.pop('results')
        return results, body
def get_form(form,offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings?Limit=100000&SpecificationForm={form}'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    if res.status_code == 500:
        return get_form(form,offset=0)
    else:
        body = res.json()
        results = body.pop('results')

        return results, body
def get_filer(filer_nid):
    """ Get one filer
    """
    url = f'{BASE_URL}/filer/v101/filers?'

    res = requests.get(url, params={ **PARAMS, 'filerNid': filer_nid }, auth=AUTH)
    if res.status_code == 500:
        return get_filer(filer_nid)
    else:
        body = res.json()

        return body['results']
def list_filers():
    """ Get all the elections
    """
    url = f'{BASE_URL}/filer/v101/filers?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    if res.status_code == 500:
        print('ping')
        return list_filers()
    else:
        body = res.json()
        return body['results']
def list_elections_influences(id):
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/election-influences?Limit=100000&ElectionNid={id}'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    if res.status_code == 500:
        return list_elections_influences(id)
    else:
        body = res.json()
        return body['results']

def list_elections():
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/elections?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    if res.status_code == 500:
        return list_elections()
    else:
        body = res.json()

        return body['results']
def export_transactions(id,offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings/{id}'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    if res.status_code == 500:
        return export_transactions(id,offset=0)
    else:
        body = res.json()

        return body

In [3]:
filers_response=list_filers()
filers_response=[item for item in filers_response if item['committeeTypes'] in [['Person'],['Candidate or Officeholder'], ['Primarily Formed Measure'],['Primarily Formed Candidate']]]

In [4]:
status = [
    {
        'isTerminated': item.get('isTerminated', {}),
        # Get the latest status and conditons for epmty lists to avoid indexError
        'status': item.get('statusItemList', None)[-1]['status'] if item.get('statusItemList', None) else None,
        'filerNid': item['filerNid'],
        'Filer Name':item['filerName'],
        'Filer Type': item['committeeTypes']
    } 
    for item in filers_response
]
status_df = pd.DataFrame(status)
# status_df

In [5]:
# get city, state, and zip from Disclosure addresses
addresses=[{'addressList':item.get('addressList',{}),'filerNid':item['filerNid']} for item in filers_response]
address_dic = {'city':[],'state':[],'zip':[],'filerNid':[]}
for item in addresses:
    for address in item['addressList']:
        if 'Disclosure' in address['addressTypes']:
            address_dic['city'].append(address['city'])
            address_dic['state'].append(address['state'])
            address_dic['zip'].append(address['zip'])
            address_dic['filerNid'].append(item['filerNid'])
address_df=pd.DataFrame(address_dic)
# address_df

In [6]:
# merge on filer id
status_address_df = status_df.merge(address_df,how='left', on='filerNid')

In [7]:
# associate a filer id to a fppc id
regs=[{'fppc_id':item.get('registrations',{}).get('CA SOS',None),'filerNid':item['filerNid']} for item in filers_response]
# get all filers with officers
officers=[[item['officers'], item['filerNid']] for item in filers_response if item['officers']]
# set up dictionary
treasurer_dic={}
# loop through filers with officers and add offcier names if officer position is treasurer the key will be the filler id
for officer in officers:
    if officer[0][0]['position']=='Treasurer':
        treasurer_dic[officer[1]]=officer[0][0]['officerName']
# match the filer id key in treasurer dic with the filer ids associated with an fppc id
for key, value in treasurer_dic.items():
    for item in regs:
        if key==item['filerNid']:
            # if a key matches a filer id then add treasure name to the dictionaries
            item['Treasurer']=value
# get only the cases with a treasurer
fppc_with_treasurer=[reg for reg in regs if reg.get('Treasurer', None)]
treasurer_df=pd.DataFrame(fppc_with_treasurer)
# treasurer_df

In [8]:
# I want to associate fppc ids with filer nids, I look through 410s, 501s, and filers_response.
# get ids from 410s
form410s=get_form('FPPC410')
form410s=form410s[0]
form410={'filerNid':[],'fppc_id':[]}
for form in form410s:
    form410['filerNid'].append(form['filerMeta']['filerId'])
    form410['fppc_id'].append(form.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS',None))

In [9]:
# get ids from 501s
form501s=get_form('FPPC501')
form501s=form501s[0]
for form in form501s:
    form410['filerNid'].append(form['filerMeta']['filerId'])
    form410['fppc_id'].append(form.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS',None))

In [10]:
# get ids from filer_response
for item in filers_response:
    form410['filerNid'].append(item.get('filerNid',{}))
    form410['fppc_id'].append(item.get('registrations',{}).get('CA SOS',None))
df_410=pd.DataFrame(form410)
df_410.drop_duplicates(inplace=True)
# In this data filer id can be associated with an ffpc id, a null value, or 'pending', I filter it for the best results then concatenate the best results last
# then drop duplicates keeping last since I put the prefferable rows last
# drop null values
t2=df_410.dropna()
# drop pending, this now only has preferable rows
t1=t2[t2['fppc_id'] != 'Pending']
duped=pd.concat([df_410, t2, t1], ignore_index=True)
best=duped.drop_duplicates(subset=['filerNid'],keep='last',inplace=False)
# best

In [11]:
# list all elections
elections=list_elections()
elections

[{'electionNid': '210326313',
  'aid': 'COAK',
  'electionDate': '2026-11-03',
  'electionCaption': '11/3/2026 - General',
  'isPublic': True,
  'electionCodes': 'General',
  'measures': [],
  'seats': [{'seatNid': '210326317',
    'electionNid': '210326313',
    'seatOfficeNid': '121709679',
    'seatCaption': 'City Auditor - City of Oakland',
    'officeName': 'City Auditor - City of Oakland'},
   {'seatNid': '210326329',
    'electionNid': '210326313',
    'seatOfficeNid': '121709698',
    'seatCaption': 'City Council - City of Oakland - 2',
    'officeName': 'City Council - City of Oakland - 2'},
   {'seatNid': '210326366',
    'electionNid': '210326313',
    'seatOfficeNid': '121709717',
    'seatCaption': 'City Council - City of Oakland - 4',
    'officeName': 'City Council - City of Oakland - 4'},
   {'seatNid': '210326368',
    'electionNid': '210326313',
    'seatOfficeNid': '121709771',
    'seatCaption': 'City Council - City of Oakland - 6',
    'officeName': 'City Council -

In [12]:
# from the list_elections response we ... 
election_list=[]
previous_df=pd.DataFrame()
for election in elections:
    # collect 
    candidates=election['candidates']
    seats=election['seats']
    election_name=election['electionCaption']
    electionNid=election['electionNid']
    election_year=election['electionDate'][:4] # get the year from the four first character, the format is yyyy-mm-dd
    election_key={'election_name':election_name, 'electionNid':electionNid, 'election year':election_year}
    election_list.append(election_key)
    if candidates and seats:
        seat_df=pd.DataFrame(seats)
        candidate_df=pd.DataFrame(candidates)
        merge_df=candidate_df.merge(seat_df, on='seatNid')
        current_df=merge_df[['candidateNid','candidateName','seatNid','officeName','electionNid','isIncumbent','isWinner']]
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
election_df=pd.DataFrame(election_list)
final_df=previous_df.merge(election_df, on='electionNid')
final_df.tail()

,candidateNid,candidateName,seatNid,officeName,electionNid,isIncumbent,isWinner,election_name,election year
366,121707619,"Kernighan, Patricia",149465894,City Council - City of Oakland - 2,125693758,False,False,6/6/2006 - Special,2006
367,133250767,"Drake, Pamela",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
368,121710764,"Kakishiba, David",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
369,121707619,"Kernighan, Patricia",133250723,City Council - City of Oakland - 2,133250667,False,False,5/17/2005 - Special,2005
370,121710764,"Kakishiba, David",149888803,Director - Oakland Unified School District - 2,149888771,True,False,11/24/2002 - General,2002


In [13]:
candidate_df

,candidateNid,candidateName,seatNid,isIncumbent,isWinner,partyAffiliationId,partyAffiliationName
0,121710764,"Kakishiba, David",149888803,True,False,0,


In [15]:
election_ids=list(set(final_df['electionNid'].to_list()))
previous_df=pd.DataFrame()
for id in election_ids:
    influences=list_elections_influences(id)
    influences_dic={'filerNid': [],'electionNid': [],'seatNid': [],'candidateNid': [],'committeeName':[],'election_name': []}
    for candidate in influences:   
        influences_dic['filerNid'].append(candidate.get('filerNid', 'None'))
        influences_dic['election_name'].append(candidate.get('electionCaption', 'None'))
        influences_dic['committeeName'].append(candidate.get('committeeName', 'None'))
        influences_dic['electionNid'].append(candidate.get('electionNid', 'None'))
        influences_dic['seatNid'].append(candidate.get('seatNid', 'None'))
        influences_dic['candidateNid'].append(candidate.get('candidateNid', 'None'))
        current_df=pd.DataFrame(influences_dic)
        current_df=current_df
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)

In [16]:
df3=previous_df
df3

,filerNid,electionNid,seatNid,candidateNid,committeeName,election_name
0,208728735,208728634,208728636,208728667,Michael Houston for Oakland City Auditor 2024,3/5/2024 - Special
1,208728735,208728634,208728636,208728667,Michael Houston for Oakland City Auditor 2024,3/5/2024 - Special
2,209723026,208728634,None,None,Nikki F Bas for Supervisor 2024,3/5/2024 - Special
3,152470251,152889535,None,0,Oaklanders for Good Government & Safe Neighbor...,11/4/2014 - General
4,152470251,152889535,None,0,Oaklanders for Good Government & Safe Neighbor...,11/4/2014 - General
...,...,...,...,...,...,...
5624,202798745,178981824,None,None,"Oaklanders Together, a coalition of small and ...",11/8/2022 - General
5625,185276534,178981824,185276351,152070991,Re-elect Nikki F Bas for City Council 2022 Off...,11/8/2022 - General
5626,204606835,178981824,None,None,Oakland Rising Ballot Committee supporting Mea...,11/8/2022 - General
5627,208909119,178981824,182704304,171131353,Sheng Thao for Oakland Mayor 2022 Officeholder...,11/8/2022 - General


In [22]:
dfNew = final_df.merge(df3,how='left', on=['candidateNid','election_name','electionNid','seatNid'])

In [18]:
core_df=dfNew[['candidateName','officeName','committeeName','election_name','filerNid','election year']] 
df4=core_df.merge(best, how='left',on=['filerNid']).drop_duplicates(ignore_index=True)

In [19]:
# merge treasurer with fppc id and filer id as keys
df5 = df4.merge(treasurer_df, how='left',on=['fppc_id','filerNid'])
# merge status and location with filer id as key
df6 = df5.merge(status_address_df, how='left',on=['filerNid'])
# rename columns for consistent style
df6.columns = ['candidate_name','office_name', 'committee_name', 'election_name', 'filler_nid', 'election_year', 'fppc_id', 'treasurer', 'is_terminated', 'status', 'filer_name', 'filer_type', 'city', 'state', 'zip']
df6[:0].to_csv('headers.csv')

In [20]:
# output table as csv
df6.to_csv('output/output.csv')
df6

,candidate_name,office_name,committee_name,election_name,filler_nid,election_year,fppc_id,treasurer,is_terminated,status,filer_name,filer_type,city,state,zip
0,"Afolabi, Baba",City Council - City of Oakland - 3,Baba Afolabi for Oakland City Council 2024,11/5/2024 - General,210517029,2024,1467002,"Eldridge, Jordan",False,ACTIVE,Baba Afolabi for Oakland City Council 2024,[Candidate or Officeholder],San Jose,CA,95110
1,"Dunbar, Donnel C",City Council - City of Oakland - 3,"Dunbar, Donnel C",11/5/2024 - General,208539747,2024,None,NaN,False,ACTIVE,"Dunbar, Donnel C",[Person],Oakland,CA,94607
2,"Fife, Carroll",City Council - City of Oakland - 3,Carroll Fife for City Council 2024,11/5/2024 - General,208660692,2024,1463441,"Fife, Carroll",False,ACTIVE,Carroll Fife for City Council 2024,[Candidate or Officeholder],Lakeport,CA,95453
3,"Hirsch, Shan M",City Council - City of Oakland - 3,Committee to Elect Shan Hirsch for City Council,11/5/2024 - General,208555064,2024,1464290,NaN,False,ACTIVE,Committee to Elect Shan Hirsch for City Council,[Candidate or Officeholder],Oakland,CA,94607
4,"Ikharo, Tariq",City Council - City of Oakland - 3,"Ikharo, Tariq",11/5/2024 - General,210906813,2024,Pending,"Ikharo, Tariq",False,ACTIVE,Tariq Ikharo for Oakland City Council 2024,[Candidate or Officeholder],Oakland,CA,94612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,"Kernighan, Patricia",City Council - City of Oakland - 2,NaN,6/6/2006 - Special,NaN,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,"Drake, Pamela",City Council - City of Oakland - 2,NaN,5/17/2005 - Special,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,"Kakishiba, David",City Council - City of Oakland - 2,NaN,5/17/2005 - Special,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,"Kernighan, Patricia",City Council - City of Oakland - 2,NaN,5/17/2005 - Special,NaN,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
